<a href="https://colab.research.google.com/github/dev-S-t/ML/blob/main/Assignment_Yardstick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
doc_paths = "/content/drive/MyDrive/mental health assignment/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install "unstructured[pdf]"


In [ ]:
!pip install \
  langchain_community \
  langchain_pinecone \
  langchain_openai \
  unstructured \
  langchain-text-splitters

In [2]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob

In [3]:
loader = DirectoryLoader(doc_paths, glob="**/*.pdf")

In [4]:
docs = loader.load()

In [ ]:
docs[0]

In [6]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [13]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

index_name = "mental-health"

# Split our documents into chunks
text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(docs)

In [ ]:
split_docs[25]

In [ ]:
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)

In [ ]:
query = "Anxiety?"

similar_docs = vectorstore.similarity_search(query)

In [ ]:
similar_docs

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="GPT-4o mini",
    temperature=0.1
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)